<a href="https://colab.research.google.com/github/Kim-TaeKyoung/ai_championship/blob/main/gaussian%2Bblur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import cv2
import numpy as np
import os 

# JSON의 모든 데이터 내 Scene을 비트 마스크의 형태로 변환한 Dictionary를 반환함
def unjson(target_json):
  return_dict = {}

  for key in target_json.keys():
    scenes = target_json[key]['scenes']
    list_max = int(target_json[key]['duration']) + 1

    return_dict.update({key : [0] * list_max})
    return_list = return_dict[key]

    for scene in scenes:
      for pivot in range(scene[0], scene[1] + 1):
        return_list[pivot] = 1

  return return_dict

path = '/content/drive/MyDrive/Colab/AI_ChampionShip/submission/inference-20211101-093158-0.1-0.1-0.1-final.json'

def cycle(blur_size):
  with open(path, 'r') as f:
    tmp = json.load(f)

  tmp_unjson = unjson(tmp)
  backup_json = tmp_unjson

  for key in tmp_unjson.keys():
    now = tmp_unjson[key]
    now = np.array(now).astype(np.float32)

    print(key)
    print(now)
    blured = np.transpose(cv2.GaussianBlur(now, (0, 0), blur_size, 0))[0]
    blured = cv2.threshold(blured, 0.8, 1, cv2.THRESH_BINARY)
    backup_json[key] = np.transpose(blured[1]).astype('uint8').tolist()[0]

  print(backup_json)

  # Post-processing
  # State machine for submission format

  converted_dict = {}

  def start_new_state(curr_list, i):
    curr_list.append(i)

  def end_current_state(curr_list, full_list, i):
    curr_list.append(i)
    full_list.append(curr_list)

  def end_state(curr_list, full_list):
    full_list.append(curr_list)

  for strings, predictions in backup_json.items():
    print(strings, predictions)
    
    previous_state = 0
    current_list = []
    replacement_list = []
    
    for i, p in enumerate(predictions):
      current_state = p

      if previous_state == 0:
        if current_state == 0:
          # Ignore non-fire state
          continue
        elif current_state == 1:
          start_new_state(current_list, i)
          previous_state = current_state
      
      if previous_state == 1:
        if current_state == 0:
            end_current_state(current_list, replacement_list, i)
            previous_state = current_state
            current_list = []
        elif current_state == 1:
          # Ignore fire state
          continue

      if i + 1 == len(predictions):
        if previous_state == 0:
          continue
        elif previous_state == 1:
          end_state(current_list, replacement_list)
          

      converted_dict.update({strings : replacement_list})
      len(converted_dict)

  import datetime
  # Replace list indices to corresponding filenames
  with open('/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/sample_fire_scenes.json', 'r') as f:
    submit = json.load(f)

  for key in converted_dict:
    submit[key]['scenes'] = converted_dict[key]

  print(json.dumps(submit, sort_keys=False, indent=2))
  nowtime = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

  with open('/content/drive/MyDrive/Colab/AI_ChampionShip/submission/' + 'gaussian_' + str(blur_size) + '_' + os.path.basename(path), 'w') as f:
    json.dump(submit, f, indent=2)

In [ ]:
cycle(3)